In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import backend as K


def exp_cam(image_model,image_array,y_pred,last_layer,num_layer): 
    
    ## y_pred-----> predicted integer label for corresponding image array
    ## last_layer-----> name of the last convolutional layer
    ## num_layer-----> filter number of last convolutional layer
    
    m_image,heat_map=[],[]
    image_file=image_array
    img_model=image_model
    pred=y_pred
    for i in range(image_file.shape[0]): 
        
        img=image_file[i]

        last_conv_layer=img_model.get_layer(last_layer)

        grads = K.gradients(img_model.output[:,pred[i]],last_conv_layer.output)[0]

        pooled_grads = K.mean(grads,axis=(0,1,2))
        iterate = K.function([img_model.input],[pooled_grads,last_conv_layer.output[0]])
        pooled_grads_value,conv_layer_output = iterate([img.reshape(1,224,224,3)])

        for i in range(num_layer):
            conv_layer_output[:,:,i] *= pooled_grads_value[i]
        heatmap = np.mean(conv_layer_output,axis=-1)

        for x in range(heatmap.shape[0]):
            for y in range(heatmap.shape[1]):
                heatmap[x,y] = np.max(heatmap[x,y],0)

        heatmap = np.maximum(heatmap,0)   ## generate heatmap with positive value
        heatmap /= np.max(heatmap)

        upsample = cv2.resize(heatmap,(224,224))  ## reshape the heatmap
        upsample=np.array(upsample)
        
        heat_map.append(upsample)
    
    
    
   
        mask_img=np.zeros((224,224,3)) ## create a null image matrix 
#         h_map=heat_map[f]
#         image1=image_file[f]
        for h in range(image_file.shape[1]):  ## highlight the corresponding image portion if output of heatmap >0
            for w in range(image_file.shape[1]):
                if upsample[h][w]>0:
                    mask_img[h][w]=img[h][w]
                else:
                     mask_img[h][w]=np.array([0,0,0])   ### otherwise mask the image
        m_image.append(mask_img)
   
    return m_image,heat_map



Using TensorFlow backend.
